# Cоздаем таблицы в clickhouse

Подключаем клиента

In [1]:
from clickhouse_driver import Client

client = Client(host='localhost')

Next, we need to create a table using the Kafka engine to connect to the topic and read data.  The engine will read from the broker at host kafka using topic ‘readings’ and a consumer group name ‘readings consumer_group1’. The input format is CSV.  Note that we omit the ‘date’ column. It’s an alias in the target table that will populate automatically from the ‘time’ column.

In [2]:
create_kafka_engine = """
CREATE TABLE readings_queue (
    readings_id Int32,
    time DateTime,
    temperature Decimal(5,2)
)
ENGINE = Kafka
SETTINGS kafka_broker_list = 'broker:29092',
       kafka_topic_list = 'readings',
       kafka_group_name = 'readings_consumer_group1',
       kafka_format = 'CSV',
       kafka_max_block_size = 1048576;
"""
client.execute(create_kafka_engine)

[]

In [3]:
create_materialized_view = """
    CREATE MATERIALIZED VIEW readings_queue_mv TO default.readings AS
    SELECT readings_id, time, temperature
    FROM readings_queue;
"""
client.execute(create_materialized_view)

[]

Смотрим доступные таблицы

In [5]:
client.execute('SHOW TABLES FROM default')

[('readings',), ('readings_queue',), ('readings_queue_mv',)]

In [6]:
client.execute('SHOW TABLES FROM shard')

[('readings',)]

In [7]:
client.execute('SHOW TABLES FROM replica')

[('readings',)]

Проверяем что сейчас данных нет

In [8]:
get_data_from_clickhouse = """
SELECT * FROM default.readings
"""

client.execute(get_data_from_clickhouse)

[]

Чтобы перейти далее, надо добавить данные в брокер сообщений. Смотри README:
Заходим внуть контейнера с портом 9092

```bash
docker ps
docker exec -it 1ca379db1a67 bash
```

В нем отправляем данные в броке

kafka-console-producer --broker-list broker:9092 --topic readings <<END
4,"2020-05-16 23:55:44",14.2
5,"2020-05-16 23:55:45",20.1
6,"2020-05-16 23:55:51",12.9
END


In [9]:
get_data_from_clickhouse = """
SELECT * FROM readings
"""

client.execute(get_data_from_clickhouse)

[(3, datetime.datetime(2020, 5, 16, 23, 55, 51), Decimal('12.9')),
 (1, datetime.datetime(2020, 5, 16, 23, 55, 44), Decimal('14.2')),
 (2, datetime.datetime(2020, 5, 16, 23, 55, 45), Decimal('20.1'))]

In [1]:
client.execute('DROP TABLE  readings_queue_mv')
client.execute('DROP TABLE  readings_queue')
client.execute('DROP TABLE  readings')

NameError: name 'client' is not defined